## Set things up

In [1]:
%matplotlib inline
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor  # not sure I'm going to use this
from sklearn.model_selection import GridSearchCV

In [2]:
RAW_DATA_SOURCE = "C:/Users/sbranchett/Data/WiFi_data/WiFiAccessPoint.csv"
CLEAN_DATA_SOURCE = "CleanedWiFiAccessPoint.csv"

## Read in data

In [3]:
def load_wifi_data(path):
    """
    Read WiFi clientCount .csv file and sum the clientCounts over building and timestamp, bucketed to 5 minutes
    
    Input: filepath the .csv file
    Output: Dataframe with "building", "time_bucket", "clientCount"
    Columns "building" generated from "locationHierarchy", and "time_bucket" generated from "timestamp"
    """
    all_data = pd.read_csv(path, delimiter=",")
    all_data = all_data.rename(columns=lambda x: x.strip())  # get rid of extra spaces in column names

    sum_clientCounts = all_data["clientCount"].sum()
    
    # extract building from 'locationHierarchy' string and deal with unknown buildings
    all_data["building"] = all_data["locationHierarchy"].str.split(" > ",expand=True)[1]
    all_data["building"] = all_data["building"].fillna("Unknown")

    # convert timestamp from epoch milliseconds to 5 minute buckets (1000 milliseconds * 60 seconds * 5 minutes)
    all_data["time_bucket"] = all_data["timestamp"].apply(lambda d: 300000*int(d/300000))
    
    # keep only the interesting columns
    all_data = all_data[["time_bucket", "building", "clientCount"]]
    
    # sum data over buildings and time buckets
    all_data = all_data.groupby(["time_bucket", "building"]).sum()  # sort on time first so that new data keeps sequence
    
    assert sum_clientCounts == all_data["clientCount"].sum()  # check that no clientCounts went missing

    return all_data

# if possible read in the cleaned data, otherwise clean the raw data and save
if os.path.isfile(CLEAN_DATA_SOURCE):
    all_data = pd.read_csv(CLEAN_DATA_SOURCE, delimiter=",")
else:
    all_data = load_wifi_data(RAW_DATA_SOURCE)
    all_data.to_csv(CLEAN_DATA_SOURCE)

print(all_data["clientCount"].sum())  # should be 3597531

3597531


In [4]:
all_data = all_data.reset_index().drop("index", axis=1)  # get rid of multiindex for StratifiedShuffleSplit
print(all_data)

         time_bucket               building  clientCount
0      1620991800000      03-Science Center           34
1      1620991800000             05-TNW-BIO            1
2      1620991800000             08-BK-City            2
3      1620991800000   19-Studuitzendbureau           13
4      1620991800000                20-Aula           32
...              ...                    ...          ...
78161  1621608600000                 64-HSL           10
78162  1621608600000          66-Fellowship          126
78163  1621608600000  Katalyse Labaratorium            8
78164  1621608600000                Unknown           32
78165  1621608600000           VLL-LAB(TNO)           11

[78166 rows x 3 columns]


## Separate Test Set stratified over buildings

In [5]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(all_data, all_data["building"]):
    strat_train_set = all_data.loc[train_index]
    strat_test_set = all_data.loc[test_index]

print(strat_train_set)

         time_bucket                           building  clientCount
11688  1621083900000                    37-Sportcentrum           15
61351  1621476000000                         36-ESP-Lab            1
55780  1621431900000                      66-Fellowship          101
26158  1621198200000                          32-OCP-IO           12
34879  1621266900000                             64-HSL           16
...              ...                                ...          ...
50892  1621393500000                            28- WNI            4
33498  1621256100000  36-EWI LB_K t/m 3 & HB_K  t/m 2e           247
4160   1621024500000                      35-Drebbelweg            2
56082  1621434300000                          63-Simona            8
55786  1621432200000                         08-BK-City          670

[62532 rows x 3 columns]


## Create Work days and Student days

In [6]:
uni_hols = (datetime.date(year=2021, month=5, day=5),
            datetime.date(year=2021, month=5, day=13),
            datetime.date(year=2021, month=5, day=14),
            datetime.date(year=2021, month=5, day=24)
           )  # National Holidays and Collective Free days

def roster(date):
    # categorise the days of the academic year at TU Delft between 1 May 2021 and 29 August 2021
    
    if (date > datetime.date(year=2021, month=5, day=16)) and \
       (date < datetime.date(year=2021, month=5, day=22)):
        categorie = "Exam_BSc"
    elif (date > datetime.date(year=2021, month=6, day=15)) and \
         (date < datetime.date(year=2021, month=6, day=19)):
        categorie = "Study_mixed"
    elif (date > datetime.date(year=2021, month=6, day=22)) and \
         (date < datetime.date(year=2021, month=6, day=26)):
        categorie = "Exam_mixed"
    elif (date > datetime.date(year=2021, month=6, day=29)) and \
         (date < datetime.date(year=2021, month=7, day=3)):
        categorie = "Exam"
    elif (date > datetime.date(year=2021, month=8, day=8)) and \
         (date < datetime.date(year=2021, month=8, day=14)):
        categorie = "Exam"
    elif (date > datetime.date(year=2021, month=7, day=4)):
        categorie = "Free"
    else:
        categorie = "Learn"
    return categorie

## Separate predictions and labels

In [7]:
crowd = strat_train_set.drop("clientCount", axis=1)
crowd_labels = strat_train_set["clientCount"].copy()
print(crowd)

         time_bucket                           building
11688  1621083900000                    37-Sportcentrum
61351  1621476000000                         36-ESP-Lab
55780  1621431900000                      66-Fellowship
26158  1621198200000                          32-OCP-IO
34879  1621266900000                             64-HSL
...              ...                                ...
50892  1621393500000                            28- WNI
33498  1621256100000  36-EWI LB_K t/m 3 & HB_K  t/m 2e 
4160   1621024500000                      35-Drebbelweg
56082  1621434300000                          63-Simona
55786  1621432200000                         08-BK-City

[62532 rows x 2 columns]


In [8]:
assert crowd.isnull().values.any() == False  # check there are no missing values

## Build Transformers

In [9]:
# specify which extra attributes to add to the data
# want to treat these as hyperparameters for grid search
add_day_of_week=False
add_time_of_day=False
add_uni_hols=False
add_weekend=False
add_academic_yr=False

In [10]:
# Set up categories for OneHotEncoder.
# This is needed later for the transformation of small sets that do not contain all the categories.

buildings = list(set(all_data["building"].values))
rosters = ["Exam_BSc", "Study_mixed", "Exam_mixed", "Exam", "Free", "Learn"]  # get a list of all roster categories, SEB could be improved

In [11]:
# Combined the Adder and Transformer so that Grid Search could switch the attributes on and off
class CombinedAdderAndTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, add_day_of_week=True, add_time_of_day=True, add_uni_hols=True, add_weekend=True, add_academic_yr=True,
                 buildings=None, rosters=None):
        self.add_day_of_week = add_day_of_week
        self.add_time_of_day = add_time_of_day
        self.add_uni_hols = add_uni_hols
        self.add_weekend = add_weekend
        self.add_academic_yr = add_academic_yr
        self.buildings = buildings
        self.rosters = rosters

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_temp = X.copy()
        num_attribs = ["time_bucket"]
        cat_attribs = ["building"]
        all_categories = [self.buildings]
        if self.add_day_of_week:
            X_temp["day_of_week"] = X["time_bucket"].apply(lambda d: datetime.datetime.fromtimestamp(d/1000).weekday())
            num_attribs.extend(["day_of_week"])
        if self.add_time_of_day:
            X_temp["time_of_day"] = X["time_bucket"].apply(lambda d: int(d%86400000))
            num_attribs.extend(["time_of_day"])
        if self.add_uni_hols:
            X_temp["hols"] = X["time_bucket"].apply(lambda d: 1. if datetime.datetime.fromtimestamp(d/1000).date() in uni_hols else 0.)
            num_attribs.extend(["hols"])
        if self.add_weekend:
            X_temp["weekend"] = X["time_bucket"].apply(lambda d: 1. if datetime.datetime.fromtimestamp(d/1000).weekday() > 4 else 0.)
            num_attribs.extend(["weekend"])
        if self.add_academic_yr:
            X_temp["academic_yr"] = X["time_bucket"].apply(lambda d: roster(datetime.datetime.fromtimestamp(d/1000).date()))
            all_categories.append(rosters)
            cat_attribs.extend(["academic_yr"])


        # combine the pipelines based on column type
        coltrans = ColumnTransformer([("num", StandardScaler(), num_attribs),
                                      ("cat", OneHotEncoder(categories=all_categories), cat_attribs)
                                     ])
        X_trans = coltrans.fit_transform(X_temp)
        return X_trans

# Build the Adder, Transformer, Linear Regression pipeline

In [12]:
full_pipeline = Pipeline([
    ('add_attributes', CombinedAdderAndTransformer(add_day_of_week, add_time_of_day, add_uni_hols, add_weekend,
                                                   add_academic_yr, buildings=buildings, rosters=rosters)),
    ('lin_reg',  LinearRegression()),
])

# print(full_pipeline.named_steps['add_attributes'])

In [13]:
# check it works
crowd_fitted = full_pipeline.fit(crowd, crowd_labels)
print(crowd_fitted)

Pipeline(steps=[('add_attributes',
                 CombinedAdderAndTransformer(add_academic_yr=False,
                                             add_day_of_week=False,
                                             add_time_of_day=False,
                                             add_uni_hols=False,
                                             add_weekend=False,
                                             buildings=['20-Aula', '23-CITG',
                                                        '25-GreenVillage',
                                                        '19-Studuitzendbureau',
                                                        '32a- Learninglab',
                                                        '37-Sportcentrum',
                                                        '30-O S',
                                                        '38-Cultureel Centrum',
                                                        '63-Simona', '21-BTUD',
                      

# Do a Grid Search

In [14]:
param_grid = {'add_attributes__add_day_of_week': [True, False], 'add_attributes__add_time_of_day': [True, False],
              'add_attributes__add_uni_hols': [True, False],
              'add_attributes__add_weekend': [True, False], 'add_attributes__add_academic_yr': [True, False],
              'add_attributes__buildings': [buildings], 'add_attributes__rosters': [rosters]}


grid_search = GridSearchCV(full_pipeline, param_grid, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(crowd, crowd_labels)

GridSearchCV(estimator=Pipeline(steps=[('add_attributes',
                                        CombinedAdderAndTransformer(add_academic_yr=False,
                                                                    add_day_of_week=False,
                                                                    add_time_of_day=False,
                                                                    add_uni_hols=False,
                                                                    add_weekend=False,
                                                                    buildings=['20-Aula',
                                                                               '23-CITG',
                                                                               '25-GreenVillage',
                                                                               '19-Studuitzendbureau',
                                                                               '32a- '
                          

In [15]:
grid_search.best_params_

{'add_attributes__add_academic_yr': True,
 'add_attributes__add_day_of_week': True,
 'add_attributes__add_time_of_day': True,
 'add_attributes__add_uni_hols': True,
 'add_attributes__add_weekend': True,
 'add_attributes__buildings': ['20-Aula',
  '23-CITG',
  '25-GreenVillage',
  '19-Studuitzendbureau',
  '32a- Learninglab',
  '37-Sportcentrum',
  '30-O S',
  '38-Cultureel Centrum',
  '63-Simona',
  '21-BTUD',
  'Katalyse Labaratorium',
  '30-IKC_ISD-FMVG',
  '50-TNW-RID',
  '46-P E lab',
  '62-LR',
  '36-EWI-HB',
  '22-TNW-TN',
  '45-LSL',
  '28- WNI',
  '36-EWI LB_K t/m 3 & HB_K  t/m 2e ',
  '31-TBM',
  '64-HSL',
  '66-Fellowship',
  '03-Science Center',
  '60-LMS',
  'VLL-LAB(TNO)',
  '33-Pulse',
  '61-Vliegtuighal',
  '35-Drebbelweg',
  '36-ESP-Lab',
  '43-EGM',
  '26-Bouwcampus',
  'Unknown',
  '58-TNW-Zuid',
  '08-BK-City',
  '32-OCP-IO',
  '34-3ME',
  '05-TNW-BIO'],
 'add_attributes__rosters': ['Exam_BSc',
  'Study_mixed',
  'Exam_mixed',
  'Exam',
  'Free',
  'Learn']}

In [16]:
grid_search.best_estimator_

Pipeline(steps=[('add_attributes',
                 CombinedAdderAndTransformer(buildings=['20-Aula', '23-CITG',
                                                        '25-GreenVillage',
                                                        '19-Studuitzendbureau',
                                                        '32a- Learninglab',
                                                        '37-Sportcentrum',
                                                        '30-O S',
                                                        '38-Cultureel Centrum',
                                                        '63-Simona', '21-BTUD',
                                                        'Katalyse Labaratorium',
                                                        '30-IKC_ISD-FMVG',
                                                        '50-TNW-RID',
                                                        '46-P E lab', '62-LR',
                                                  

In [17]:
# take a look at the different permutations
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params['add_attributes__add_academic_yr'], params['add_attributes__add_day_of_week'],
          params['add_attributes__add_time_of_day'], params['add_attributes__add_uni_hols'],
          params['add_attributes__add_weekend'])

97.44038970891434 True True True True True
97.45137132742174 True True True True False
97.45121568792788 True True True False True
97.45142788112607 True True True False False
97.7023218653896 True True False True True
97.70954847408748 True True False True False
97.70933377488828 True True False False True
97.71094864864526 True True False False False
98.4064762390381 True False True True True
98.4056106250453 True False True True False
98.40594585200225 True False True False True
98.43231452079503 True False True False False
98.40588723741662 True False False True True
98.40502164180518 True False False True False
98.40535574543439 True False False False True
98.43230282079912 True False False False False
97.44220509309443 False True True True True
98.2178898392352 False True True True False
98.23911901905242 False True True False True
98.23858887535914 False True True False False
97.70220517073992 False True False True True
98.22136494288564 False True False True False
98.2402092245

Save what we have so far:
```
from joblib import dump

dump(lin_reg, "lin_reg.joblib")
dump(tree_reg, "tree_reg.joblib")
dump(forest_reg, "forest_reg.joblib")
```

Still want to do:

* possibly add time from lunch as an attribute
* see what happens with 4 weeks of data
* write blog
* illustrate blog with nice graphs

See also: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html